# X-Ray Landmark Detection on Google Colab

## Preperation

### Imports and installation of the required libraries

The libraries tensorboardx and bayesian-optimization are not within the virtual environment of Google Colab, hence they have to be installed manually.

In [0]:
import glob
import os
from zipfile import ZipFile

from google.colab import drive
from google.colab import files

! pip install tensorboardx
! pip install bayesian-optimization


### Google Colab or Zip upload
Either upload your project to Google Drive and mount it or upload project manually as .zip file and extract it.

In [0]:
use_google_drive = True

if use_google_drive:
  drive.mount('gdrive')
  % cd gdrive/My\ Drive/MLMI_SS19
else:
  file = files.upload()
  file_path = os.path.join(ROOT,list(file.keys())[0])
  zip_file = ZipFile(file_path)
  zip_file.extractall(ROOT)
  zip_file.close()

### Tensorboard and tunneling
Install ngrok for tunneling 

In [0]:
if os.path.exists("ngrok-stable-linux-amd64.zip"):
  os.remove("ngrok-stable-linux-amd64.zip")

if os.path.exists("ngrok"):
  os.remove("ngrok")
  
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

Start tensorboard and forward port with ngrok

In [0]:
LOG_DIR = 'saved/log/'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

get_ipython().system_raw('./ngrok http 6006 &')

Extract ngrok url for accessing tensorboard

**Attention**: Sometimes it throws an error like this:
```
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)
```
If this is the case the easiest way to solve this issue is to delete the ngrok*.zip and ngrok from the Google Drive folder and install them again.


In [0]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

## Training

### Imports

In [0]:
from config import CONFIG  
from parse_config import ConfigParser
from train import main

### Manual Training
Modify parameters and train model **manually**

In [0]:
CONFIG['arch']['args']['x_channels'] = 256
CONFIG['arch']['args']['stage_channels'] = 512
CONFIG['arch']['args']['num_stages'] = 5
CONFIG['arch']['args']['dilation'] = 1
CONFIG['arch']['args']['depthwise_separable_convolution'] = False


CONFIG['data_loader']['args']['data_dir'] = "data/XRay/Patient_0"
CONFIG['data_loader']['args']['batch_size'] = 10
CONFIG['data_loader']['args']['validation_split'] = 0.0
CONFIG['data_loader']['args']['shuffle'] = False

CONFIG['optimizer']['args']['lr'] = 1e-5

CONFIG['trainer']['epochs'] = 1000
CONFIG['trainer']['save_period'] = 1
CONFIG['trainer']['early_stop'] = 50

CONFIG['fraction_of_dataset'] = 1
CONFIG['sigma'] = 80
CONFIG['prediction_blur'] = 2
CONFIG['threshold'] = 0.01

main(ConfigParser(CONFIG))

### Resume training
By default it takes your last training run and the last model of it. 
If you want to use a specific run or a specific model you can provide it like this:

```
run_dir = "0629_194146"
model_pth = "checkpoint-epoch11.pth"
```



In [0]:
from importlib.machinery import SourceFileLoader

base_saved_dir = "saved/models/XRay"

run_dir = None
model_pth = None

for temp_run_dir in os.listdir(base_saved_dir)[::-1]:
  if run_dir is None :
      run_path = os.path.join(base_saved_dir, temp_run_dir)
  else:
    run_path = os.path.join(base_saved_dir, run_dir)

  if model_pth is None :
    model_path_list = glob.glob(f'{run_path}/*.pth')
    if not model_path_list:
      continue
    model_path = model_path_list[-1]
    break
  else:
    model_path = os.path.join(run_path, model_pth)
    break

config = SourceFileLoader("CONFIG", os.path.join(run_path, 'config.py')).load_module().CONFIG

main(ConfigParser(config, model_path))


### Bayesian Optimization
Do **automatic** Bayesian optimization

In [0]:
from bayes_opt_train import run_bayes_opt

run_bayes_opt({
    'num_channels': (6, 8),  # {64, 128, 256}
    'num_stacks': (2, 7),
    'num_blocks': (1, 7),
    'kernel_size': (1, 4),  # {3, 5, 7, 9}
    'sigma': (0.6, 5),
    'prediction_blur': (0.01, 1),
    'threshold': (0.00001, 0.2),
    'epochs': (200, 200)

}, init_points=10, n_iter=10)